# Trabalho Final

Titulo   
Introdução   
Coleta e Carregamento de Dados   
Limpeza de Dados   
Analise Exploratória   
Funções Reutilizáveis   
Analises   
Analise 1   
Analise 2   
Insights   
Conclusão e Reflexões

### Participantes   

### Introdução

### Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Funções

In [ ]:
def import_data(path,separador = ',', encoding = 'utf-8'):
    #Importa arquivos csv
    #Parâmetros: path = caminho do arquivo, separador = separador do arquivo, encoding = codificação do arquivo
    #retorna um dataframe
    try:
        df = pd.read_csv(path, sep=separador, encoding=encoding)
        print("Arquivo importado com sucesso!")
        return df
    except UnicodeDecodeError as ue:
        print(f"Erro de codificação: {ue}. Tente outro encoding.")
    except FileNotFoundError as fnf:
        print(f"Arquivo não encontrado: {fnf}. Verifique o caminho.")
    except Exception as e:
        print(f"Erro ao importar o arquivo: {e}")


def verificar_NaN (dataframe: pd.DataFrame):
    #Verifica a quantidade de valores NaN em cada coluna do dataframe
    #Retorna colunas com NaN, a quantidade de NaN em cada coluna e o percentual de NaN em cada coluna
    quantidade_NaN = dataframe.isna().sum()
    df_NaN = quantidade_NaN.reset_index()
    df_NaN = df_NaN.rename(columns={'index':'coluna', 0:'qnt_NaN'})
    df_NaN = df_NaN[df_NaN['qnt_NaN'] > 0]
    df_NaN['percentual_NaN'] = ((df_NaN['qnt_NaN'] / len(dataframe)) * 100).round(2)
    df_NaN['percentual_NaN'] = df_NaN['percentual_NaN'].apply(lambda x: f'{x:.2f}%') 
    df_NaN = df_NaN.sort_values(by='qnt_NaN', ascending=False).reset_index(drop=True)
    return df_NaN


def verificar_duplicados (dataframe): #Corrigir
    #Verifica a quantidade de valores duplicados em cada coluna do dataframe
    #Retorna colunas com duplicados, a quantidade de duplicados em cada coluna e o percentual de duplicados em cada coluna
    quantidade_duplicados = pd.DataFrame.duplicated().sum()
    if quantidade_duplicados == 0:
        print("Não há valores duplicados no dataframe.")
    else:
        df_dup = quantidade_duplicados.reset_index()
        df_dup = df_dup.rename(columns={'index':'coluna', 0:'qnt_dup'})
        df_dup = df_dup[df_dup['qnt_dup'] > 0]
        df_dup['percentual_dup'] = ((df_dup['qnt_dup'] / len(dataframe)) * 100).round(2)
        df_dup['percentual_dup'] = df_dup['percentual_dup'].apply(lambda x: f'{x:.2f}%') 
        df_dup = df_dup.sort_values(by='qnt_dup', ascending=False).reset_index(drop=True)
        return df_dup

def gerar_metadados(dataframe):
    # Gera um dataframe com metadados do dataframe original
    # Parâmetros: dataframe = dataframe original
    # Retorna um dataframe com os metadados

    metadata = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes.values,
        'qt_nulos': dataframe.isnull().sum().values,
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe)) * 100, 2).values,
        'cardinalidade': dataframe.nunique().values,
    })

    # Não reordena as colunas
    metadata = metadata.reset_index(drop=True)

    return metadata

def verifica_dados_duplicados(dataframe):
    qtd = dataframe.duplicated().sum()
    print(f'Foram encontrados {qtd} de registros duplicados\n')
    print('Prévia dos dados')
    filtro = dataframe.duplicated()
    return dataframe[filtro]

# Função verifica se o pais esta presente em ambas as bases
def pais_presente(df1, df2, coluna_pais_df1='country', coluna_pais_df2='country'):
    #df1 dataframe base do trabalho
    #df2 dataframe adicional
    #Verifica se os países presentes na coluna especificada do df1 estão presentes na coluna especificada do df2
    #Parâmetros: df1, df2 = dataframes, coluna (nome da coluna que contém os países)
    df1[coluna_pais_df1] = df1[coluna_pais_df1].str.lower() # Converte para minúsculas para evitar problemas de case sensitive
    df2[coluna_pais_df2] = df2[coluna_pais_df2].str.lower() # Converte para minúsculas para evitar problemas de case sensitive
    paises_df1 = set(df1[coluna_pais_df1].unique()) #Unique values em formato de conjunto
    paises_df2 = set(df2[coluna_pais_df2].unique()) #Unique values em formato de conjunto
    interc = paises_df1 & paises_df2 # Interseção dos dois conjuntos
    if paises_df1 == paises_df2:
        print("Todos os países do df1 estão presentes no df2")
        return df2
    else:
        df2['interc'] = df2[coluna_pais_df2].isin(interc).astype(int)  # Cria uma nova coluna indicando se o país está presente em ambos os dataframes 1 para true e 0 para false
        df2_filtrado = df2[df2['interc'] == 1].copy() # Filtra o df2 para manter apenas os países presentes em ambos os dataframes e remove a coluna interc
        print(f"O df2 foi filtrado para conter apenas países que também estão no df1.")
        print(f"Número de países únicos na interseção: {len(interc)}")
        return df2_filtrado

        

### Coleta de dados

In [3]:
local = '../database/gapminder_full.csv'
data = import_data(local)

Arquivo importado com sucesso!


In [4]:
df = data.copy()
df.head()

,country,year,pop,continent,lifeExp,gdpPercap
0,Afghanistan,1952,8425333.0,Asia,28.801,779.445314
1,Afghanistan,1957,9240934.0,Asia,30.332,820.853030
2,Afghanistan,1962,10267083.0,Asia,31.997,853.100710
3,Afghanistan,1967,11537966.0,Asia,34.020,836.197138
4,Afghanistan,1972,13079460.0,Asia,36.088,739.981106


In [5]:
shape = df.shape
shape

(1736, 6)

In [5]:
generate_metadata(df)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,country,object,0,0.0,142
1,year,int64,0,0.0,12
2,pop,float64,0,0.0,1704
3,continent,object,0,0.0,5
4,lifeExp,float64,0,0.0,1626
5,gdpPercap,float64,0,0.0,1704


### Tratamento de dados

In [7]:
verifica_dados_duplicados(df)

Foram encontrados 32 de registros duplicados

Prévia dos dados


,country,year,pop,continent,lifeExp,gdpPercap
72,Afghanistan,1997,2.222742e+07,Asia,41.763,635.341351
145,Bolivia,1987,6.156369e+06,Americas,57.251,2753.691490
146,Bolivia,1992,6.893451e+06,Americas,59.957,2961.699694
147,Bolivia,1997,7.693188e+06,Americas,62.050,3326.143191
148,Bolivia,2002,8.445134e+06,Americas,63.883,3413.262690
149,Bolivia,2007,9.119152e+06,Americas,65.554,3822.137084
161,Belgium,2007,1.039223e+07,Europe,79.441,33692.605080
207,Burkina Faso,1987,7.586551e+06,Africa,49.557,912.063142
232,Cambodia,1987,8.371791e+06,Asia,53.914,683.895573
345,China,1997,1.230075e+09,Asia,70.426,2289.234136


In [6]:
df = df.drop_duplicates().reset_index(drop=True)

### Analisando as colunas

In [7]:
paises = df['country'].unique()
lista_paises = list(paises)

In [8]:
anos = df['year'].unique()
ano_pais = df['year'].value_counts()

In [11]:
#Conferindo se todos os paises tem registros em todos os anos
print(len(paises))
ano_pais

142


year
1952    142
1957    142
1962    142
1967    142
1972    142
1977    142
1982    142
1987    142
1992    142
1997    142
2002    142
2007    142
Name: count, dtype: int64

In [12]:
continentes = df['continent'].unique()
continentes

array(['Asia', 'Europe', 'Africa', 'Americas', 'Oceania'], dtype=object)

### Analise Descritiva

In [9]:
df.describe()

,year,pop,lifeExp,gdpPercap
count,1704.00000,1.704000e+03,1704.000000,1704.000000
mean,1979.50000,2.960121e+07,59.474439,7215.327081
std,17.26533,1.061579e+08,12.917107,9857.454543
min,1952.00000,6.001100e+04,23.599000,241.165876
25%,1965.75000,2.793664e+06,48.198000,1202.060309
50%,1979.50000,7.023596e+06,60.712500,3531.846988
75%,1993.25000,1.958522e+07,70.845500,9325.462346
max,2007.00000,1.318683e+09,82.603000,113523.132900


Bases de dados para se somar
Obs: IDH começou a ser calculado em 1990

In [10]:
teste = import_data('../database/arquivo1.csv')
teste

Arquivo importado com sucesso!


,Time,Time Code,Country Name,Country Code,Access to electricity (% of population) [EG.ELC.ACCS.ZS],"Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]","Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]",Agricultural land (sq. km) [AG.LND.AGRI.K2],Agricultural land (% of land area) [AG.LND.AGRI.ZS],Armed forces personnel (% of total labor force) [MS.MIL.TOTL.TF.ZS],...,"Mortality rate, infant (per 1,000 live births) [SP.DYN.IMRT.IN]","Mortality rate, infant, female (per 1,000 live births) [SP.DYN.IMRT.FE.IN]","Mortality rate, infant, male (per 1,000 live births) [SP.DYN.IMRT.MA.IN]",Oil rents (% of GDP) [NY.GDP.PETR.RT.ZS],"Primary education, duration (years) [SE.PRM.DURS]","Primary education, pupils (% female) [SE.PRM.ENRL.FE.ZS]","Primary education, teachers (% female) [SE.PRM.TCHR.FE.ZS]","Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age) [SH.ALC.PCAP.LI]","Total alcohol consumption per capita, female (liters of pure alcohol, projected estimates, female 15+ years of age) [SH.ALC.PCAP.FE.LI]","Total alcohol consumption per capita, male (liters of pure alcohol, projected estimates, male 15+ years of age) [SH.ALC.PCAP.MA.LI]"
0,1990,YR1990,Afghanistan,AFG,..,..,..,380400,58.3229842233568,1.94743405417493,...,144.5,138.5,150.2,..,6,34.00202,58.7448692321777,..,..,..
1,1990,YR1990,Albania,ALB,100,100,100,11210,40.9124087591241,..,...,27.2,23,31.2,4.8458773941323,4,48.07611,..,..,..,..
2,1990,YR1990,Algeria,DZA,..,..,..,386760,16.2385482882263,1.87742220990474,...,39.8,36.3,43.1,15.7969733351133,6,44.66128,39.7516288757324,..,..,..
3,1990,YR1990,American Samoa,ASM,..,..,..,19.3,9.65,..,...,..,..,..,..,8,47.10753,64.2082366943359,..,..,..
4,1990,YR1990,Andorra,AND,100,100,100,230,48.936170212766,..,...,9.1,7.7,10.3,..,5,..,..,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2929,Data from database: World Development Indicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
generate_metadata(teste)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,Time,object,3,0.10,13
1,Time Code,object,5,0.17,11
2,Country Name,object,5,0.17,266
3,Country Code,object,5,0.17,266
4,Access to electricity (% of population) [EG.EL...,object,5,0.17,874
5,"Access to electricity, rural (% of rural popul...",object,5,0.17,807
6,"Access to electricity, urban (% of urban popul...",object,5,0.17,848
7,Agricultural land (sq. km) [AG.LND.AGRI.K2],object,5,0.17,2118
8,Agricultural land (% of land area) [AG.LND.AGR...,object,5,0.17,2183
9,Armed forces personnel (% of total labor force...,object,5,0.17,2228


In [13]:
# Função verifica se o pais esta presente em ambas as bases
def pais_presente(df1, df2, coluna_pais_df1='country', coluna_pais_df2='country'):
    #df1 dataframe base do trabalho
    #df2 dataframe adicional
    #Verifica se os países presentes na coluna especificada do df1 estão presentes na coluna especificada do df2
    #Parâmetros: df1, df2 = dataframes, coluna (nome da coluna que contém os países)
    df1[coluna_pais_df1] = df1[coluna_pais_df1].str.lower() # Converte para minúsculas para evitar problemas de case sensitive
    df2[coluna_pais_df2] = df2[coluna_pais_df2].str.lower() # Converte para minúsculas para evitar problemas de case sensitive
    paises_df1 = set(df1[coluna_pais_df1].unique()) #Unique values em formato de conjunto
    paises_df2 = set(df2[coluna_pais_df2].unique()) #Unique values em formato de conjunto
    interc = paises_df1 & paises_df2 # Interseção dos dois conjuntos
    paises_ausentes_em_df2 = paises_df1 - paises_df2
    lista_ausentes = list(paises_ausentes_em_df2)
    if paises_df1 == paises_df2:
        print("Todos os países do df1 estão presentes no df2")
        return df2
    else:
        df2['interc'] = df2[coluna_pais_df2].isin(interc).astype(int)  # Cria uma nova coluna indicando se o país está presente em ambos os dataframes 1 para true e 0 para false
        df2_filtrado = df2[df2['interc'] == 1].copy() # Filtra o df2 para manter apenas os países presentes em ambos os dataframes e remove a coluna interc
        print(f"O df2 foi filtrado para conter apenas países que também estão no df1.")
        print(f"Número de países únicos na interseção: {len(interc)}")
        print(f"Foram encontrados {lista_ausentes} países em df1 que não estão em df2.")
        return df2_filtrado

        

In [14]:
teste = pais_presente(df, teste, 'country', 'Country Name')

O df2 foi filtrado para conter apenas países que também estão no df1.
Número de países únicos na interseção: 123
Foram encontrados ['yemen rep.', 'somalia', 'korea dem. rep.', 'hong kong china', 'vietnam', 'czech republic', 'congo dem. rep.', 'iran', 'turkey', 'congo rep.', 'taiwan', 'egypt', 'syria', 'swaziland', 'reunion', 'korea rep.', 'gambia', 'venezuela', 'puerto rico'] países em df1 que não estão em df2.


In [30]:
teste

,Time,Time Code,Country Name,Country Code,Access to electricity (% of population) [EG.ELC.ACCS.ZS],"Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]","Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]",Agricultural land (sq. km) [AG.LND.AGRI.K2],Agricultural land (% of land area) [AG.LND.AGRI.ZS],Armed forces personnel (% of total labor force) [MS.MIL.TOTL.TF.ZS],...,"Mortality rate, infant, female (per 1,000 live births) [SP.DYN.IMRT.FE.IN]","Mortality rate, infant, male (per 1,000 live births) [SP.DYN.IMRT.MA.IN]",Oil rents (% of GDP) [NY.GDP.PETR.RT.ZS],"Primary education, duration (years) [SE.PRM.DURS]","Primary education, pupils (% female) [SE.PRM.ENRL.FE.ZS]","Primary education, teachers (% female) [SE.PRM.TCHR.FE.ZS]","Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age) [SH.ALC.PCAP.LI]","Total alcohol consumption per capita, female (liters of pure alcohol, projected estimates, female 15+ years of age) [SH.ALC.PCAP.FE.LI]","Total alcohol consumption per capita, male (liters of pure alcohol, projected estimates, male 15+ years of age) [SH.ALC.PCAP.MA.LI]",interc
0,1990,YR1990,afghanistan,AFG,..,..,..,380400,58.3229842233568,1.94743405417493,...,138.5,150.2,..,6,34.00202,58.7448692321777,..,..,..,1
1,1990,YR1990,albania,ALB,100,100,100,11210,40.9124087591241,..,...,23,31.2,4.8458773941323,4,48.07611,..,..,..,..,1
2,1990,YR1990,algeria,DZA,..,..,..,386760,16.2385482882263,1.87742220990474,...,36.3,43.1,15.7969733351133,6,44.66128,39.7516288757324,..,..,..,1
5,1990,YR1990,angola,AGO,..,..,..,444560,35.6589396005454,2.33513632932001,...,112.9,131.1,28.0268256543238,4,48.02435,..,..,..,..,1
7,1990,YR1990,argentina,ARG,92.1548004150391,78.8945883201315,94.1390151977539,1275650,46.6128790619325,0.631923371039086,...,22,27.5,2.18272104761879,7,49.09786,..,..,..,..,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2866,2000,YR2000,united states,USA,100,100,100,4143990,45.2305848555761,0.988718986850928,...,6.4,7.8,0.307551276869583,6,..,..,9.05,4.04,14.33,1
2867,2000,YR2000,uruguay,URY,97.6,78.1,99.3,149580,85.4645183407611,1.65509557504143,...,12.9,16.5,0,6,48.52453,..,6.97,3.21,11.12,1
2873,2000,YR2000,west bank and gaza,PSE,99.7,99.2,99.9,4682.131,77.7762624584718,..,...,24.6,28.1,0,4,48.91064,58.7219009399414,..,..,..,1
2875,2000,YR2000,zambia,ZMB,16.7,2.1,44.1,225260,30.3017258773995,0.7381445165009,...,83.2,95.2,0,7,48.07863,50.4298706054688,3.56,1.43,5.96,1


In [43]:
teste_pais = teste['Country Name'].unique()
lista_teste_pais = list(teste_pais)

In [45]:
lista_paises_m = [i.lower() for i in lista_paises]

### Graficos

#### Graficos de Linha

In [ ]:
def graficos_linhas(df, coluna:str, unico:str, observado:str, metrica):
    #rever nomes das variaveis
    #Função responsavel por construir graficos em linha
    #df = dataframe
    # coluna = Seleciona coluna pais ou continente
    #unico = unique() (páis, continente)
    # observado = parametro
    # y = ano
    filtro = df[df[coluna] == unico]
    base = filtro[[observado,'year']].metrica
    grafico = 

    

    














sns.lineplot(x = 'year', y = 'population', data = filtro)
sns.plot()

KeyError: "None of [Index(['afghanistan', 'afghanistan', 'afghanistan', 'afghanistan',\n       'afghanistan', 'afghanistan', 'afghanistan', 'afghanistan',\n       'afghanistan', 'afghanistan',\n       ...\n       'zimbabwe', 'zimbabwe', 'zimbabwe', 'zimbabwe', 'zimbabwe', 'zimbabwe',\n       'zimbabwe', 'zimbabwe', 'zimbabwe', 'zimbabwe'],\n      dtype='object', length=1704)] are in the [columns]"